# Exercise 2: Cartpole LQR Control
In this problem we consider controlling a cart-pole system, which consists of a "cart" that can travel linearly along a one-dimensional track, and has a pendulum attached to it.
The objective is to control the cart position to stablize the pendulum about its "inverted" position.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from cartpole_utils import *

#### Exercise 2.1: Linearizing the Cartpole Dynamics
Implement the function `compute_lti_matrices` to compute the linear time-invariant (LTI) system matrices $A$ and $B$ for the cart-pole system about the upright equilibrium state.

In [ ]:
def compute_lti_matrices(cartpole: CartPole, dt: float) -> tuple[np.ndarray, np.ndarray]:
    """
    Compute the linearized dynamics matrices A and B of the LTI system

    Args:
        cartpole: CartPole object storing params
        dt: time step

    Returns:
        tuple[np.ndarray, np.ndarray]: Tuple of:
            np.ndarray: The A (dynamics) matrix, shape (n, n)
            np.ndarray: The B (controls) matrix, shape (n, m)
    """
    mp, mc, L, g = cartpole.mp, cartpole.mc, cartpole.L, cartpole.g
    ############################## Code starts here ##############################
    raise NotImplementedError()
    ############################## Code ends here ##############################

#### Exercise 2.2: Implement Discrete-time Riccati Equations
Implement the function `discreteLQR.solve` to solve the LQR problem by implementing the Riccati recursion to approximate the optimal gain matrix $K_\infty$ for the linearized and discrete-time cart-pole system.

In [ ]:
class discreteLQR(object):
    def __init__(self, Q, R, eps=1e-4, max_iters=1000):
        """
        Initialize the discrete-time LQR problem with cost function matrices.
    
        Args:
            Q (np.ndarray): State cost matrix, shape (n, n)
            R (np.ndarray): Control cost matrix, shape (m, m)
            eps: termination threshold for Riccati recursion
            max_iters: maximum number of Riccati recursion iterations
        """
        if max_iters <= 1:
            raise ValueError("Argument `max_iters` must be at least 1.")
        self.Q = Q
        self.R = R
        self.eps = eps
        self.max_iters = max_iters
        self.n = Q.shape[0]  # state dimension
        self.m = R.shape[0]  # control dimension
        
        # Initialize gain matrix
        self.K = np.zeros((self.m, self.n))

        # Initialize the reference points
        self.u_ref = np.zeros(self.m)
        self.s_ref = np.zeros(self.n)

    def solve(
        self, A: np.ndarray, B: np.ndarray, s_ref: np.ndarray, u_ref: np.ndarray
    ) -> np.ndarray:
        """
        Compute the gain matrix K through Ricatti recursion. Stores the gain matrix
        in self.K and returns it.
    
        Args:
            A (np.ndarray): Dynamics matrix, shape (n, n)
            B (np.ndarray): Controls matrix, shape (n, m)
            s_ref (np.ndarray): Reference trajectory that A, B are linearized about, shape(num_timesteps, n)
                                Note, every point of the reference trajectory must have the same A, B linearization.
            u_ref (np.ndarray): Reference control that A, B are linearized about, shape(m,)
    
        Returns:
            np.ndarray: Gain matrix K, shape (m, n)
        """
        self.u_ref = u_ref
        self.s_ref = s_ref
        P_prev = np.zeros((self.n, self.n))  # initialization
        converged = False
        for i in range(self.max_iters):
            ############################## Code starts here ##############################
            raise NotImplementedError()
            ############################## Code ends here ##############################
        if not converged:
            raise RuntimeError("Ricatti recursion did not converge!")
        self.K = K
        return K
    
    def compute_control(self, k: int, s: np.ndarray, closed_loop: bool=True):
        """
        Compute the control value for the LQR controller. If `closed_loop` is false
        just return the open loop u_ref value.
    
        Arguments
        ---------
        k: current time step
        s: current state
    
        Returns
        -------
        u: control value to apply
        """
        return self.K @ (s - self.s_ref[k]) + self.u_ref if closed_loop else self.u_ref

#### Exercise 2.3: Simulate Stabilized Cartpole
Run the code below to simulate the stabilized cartpole with an LQR feedback controller defined using the functions you implemented above.

In [ ]:
# Define constants for the cart-pole system
mp = 2.0  # pendulum mass
mc = 10.0  # cart mass
L = 1.0  # pendulum length
g = 9.81  # gravitational acceleration
cartpole = CartPole(mp, mc, L, g)

# Compute discrete LTI model with 10Hz sampling rate
dt = 0.1
t = np.arange(0.0, 30.0, dt)
A, B = compute_lti_matrices(cartpole, dt)

# Compute LQR gain matrix
n = 4
m = 1
Q = np.eye(n)  # state cost matrix
R = np.eye(m)  # control cost matrix
lqr = discreteLQR(Q, R)
s_ref = np.array([0.0, np.pi, 0.0, 0.0]) * np.ones((t.size, 1)) # stablize at equilibrium
u_ref = np.array([0.0])
lqr.solve(A, B, s_ref, u_ref)

# Simulate and plot results
s0 = np.array([0.0, 3 * np.pi / 4, 0.0, 0.0])
s, u = simulate_cartpole(cartpole, t, s0, lqr)
plot_state_and_control_history(s, u, t, lqr.s_ref, "cartpole_balance")
animate_cartpole(t, s[:, 0], s[:, 1])

#### Exercise 2.4: Use LQR to Track Time-varying Reference Trajectory
Implement the function `reference` below to generate a time-varying reference trajectory that keeps the pendulum upright while moving the cart's linear position according to $\bar{x}(t) = a \sin(2\pi t/ T)$. Then, run the code below to see the simulated results. Try playing with the state penalty matrix $Q$ to see if you can improve the tracking error.

In [ ]:
def reference(t: float) -> np.ndarray:
    """
    Compute the reference state (s_bar) at time t

    Args:
        t (float): Evaluation time

    Returns:
        np.ndarray: Reference state, shape (n,)
    """
    a = 10.0  # Amplitude
    T = 10.0  # Period

    ############################## Code starts here ##############################
    raise NotImplementedError()
    ############################## Code ends here ##############################

In [ ]:
# Define constants for the cart-pole system
mp = 2.0  # pendulum mass
mc = 10.0  # cart mass
L = 1.0  # pendulum length
g = 9.81  # gravitational acceleration
cartpole = CartPole(mp, mc, L, g)

# Compute discrete LTI model with 10Hz sampling rate
dt = 0.1
A, B = compute_lti_matrices(cartpole, dt)

# Compute LQR gain matrix
n = 4
m = 1
Q = np.eye(n)  # state cost matrix
R = np.eye(m)  # control cost matrix
lqr = discreteLQR(Q, R)
s_ref = np.array([reference(ti) for ti in t])
u_ref = np.array([0.0])
lqr.solve(A, B, s_ref, u_ref)

# Simulate with time-varying reference and plot results
s0 = np.array([0.0, np.pi, 0.0, 0.0])
s, u = simulate_cartpole(cartpole, t, s0, lqr)
plot_state_and_control_history(s, u, t, lqr.s_ref, "cartpole_balance_tv")
animate_cartpole(t, s[:, 0], s[:, 1])